In [143]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from copy import copy
from math import floor

# internal imports
from pympc.geometry.polyhedron import Polyhedron
from pympc.dynamics.discrete_time_systems import LinearSystem, AffineSystem, PieceWiseAffineSystem
from pympc.control.hybrid_benchmark.controllers import HybridModelPredictiveController
from pympc.plot import plot_input_sequence, plot_state_trajectory, plot_output_trajectory
from pympc.control.hybrid_benchmark.utils import get_constraint_set, remove_redundant_inequalities_fast, convex_hull_method_fast

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Problem set-up

In [203]:
# numeric parameters of the system
m = 1.
r = .1
I = .4*m*r**2.
d = .5
k = 500.
bt = 50.
bn = 20.
mu = .2
g = 10.
h = .02

In [204]:
# state: x = (qxb, qyb, tzb, qxp, qyp, vxb, vyb, ozb)
# input: u = (vxp, vyp)

# discretization method
method = 'zero_order_hold'

# dynamics n.1 (ball in the air)
A1 = np.block([
    [np.zeros((3,5)), np.eye(3)],
    [np.zeros((5,8))]
    ])
B1 = np.block([
    [np.zeros((3,2))],
    [np.eye(2)],
    [np.zeros((3,2))]
    ])
c1 = np.array([0.]*6 + [-g, 0.])
S1 = AffineSystem.from_continuous(A1, B1, c1, h, method)

# dynamics n.2 (paddle pulling ball)
S2 = copy(S1)

# dynamics n.3 (paddle pushing ball, sticking)
# (no offset terms, the zero of the position of the ball is shifted by m*g/k-r)
A3 = copy(A1)
A3[5,5] = -bt/m
A3[5,7] = -r*bt/m
A3[6,1] = -k/m
A3[6,4] = k/m
A3[6,6] = -bn/m
A3[7,5] = -r*bt/I
A3[7,7] = -r*r*bt/I
B3 = copy(B1)
B3[5,0] = bt/m
B3[6,1] = bn/m
B3[7,0] = r*bt/I
S3 = LinearSystem.from_continuous(A3, B3, h, method)

# dynamics n.4 (paddle pushing ball, ball sliding left wrt paddle)
A4 = copy(A3)
A4[5] = mu * A4[6]
A4[7] = r*m/I * A4[5]
B4 = copy(B3)
B4[5] = mu * B4[6]
B4[7] = r*m/I * B4[5]
c4 = np.array([0.]*5 + [mu*g, 0., r*m*mu*g/I])
S4 = AffineSystem.from_continuous(A4, B4, c4, h, method)

# dynamics n.5 (paddle pushing ball, ball sliding right wrt paddle)
A5 = copy(A4)
A5[5] = -A5[5]
A5[7] = -A5[7]
B5 = copy(B4)
B5[5] = -B5[5]
B5[7] = -B5[7]
c5 = -c4
S5 = AffineSystem.from_continuous(A5, B5, c5, h, method)

# list of dynamics
S_list = [S1, S2, S3, S4, S5]

In [205]:
# state: x = (qxb, qyb, tzb, qxp, qyp, vxb, vyb, ozb)
# input: u = (vxp, vyp)

### limite sulla forza massima

# state bounds
x_max = np.array([1., .5, 3*np.pi, .5, .3, 10., 10., 20.])*2.
x_min = - x_max # np.array([-1., -1., -1.2, -.2, -5., -5.])

# input bounds
u_max = np.array([100., 100.])*2.
u_min = - u_max

# domain bounds
xu_min = np.concatenate((x_min, u_min))
xu_max = np.concatenate((x_max, u_max))

'''
domain n.1, out of contact if:
qyb >= qyp + mg/k
'''
D1 = Polyhedron.from_bounds(xu_min, xu_max)
D1.add_inequality(
    np.array([[0., -1., 0., 0., 1., 0., 0., 0., 0., 0.]]),
    np.array([-m*g/k])
)
assert D1.bounded
assert not D1.empty

'''
domain n.2, pulling if:
- qyb <= qyp + mg/k,
- fn = k(qyp-qyb) + bn(vyp-vyb) + mg <= 0
'''
D2 = Polyhedron.from_bounds(xu_min, xu_max)
D2.add_inequality(
    np.array([[0., 1., 0., 0., -1., 0., 0., 0., 0., 0.]]),
    np.array([m*g/k])
)
D2.add_inequality(
    np.array([[0., -k, 0., 0., k, 0., -bn, 0., 0., bn]]),
    np.array([-m*g])
)
assert D2.bounded
assert not D2.empty

'''
domain n.3, sticking if:
- qyb <= qyp + mg/k,
- fn = k(qyp-qyb) + bn(vyp-vyb) + mg >= 0,
- ft = bt(vxp-vxb-r ozb) <= mu fn = mu k(qyp-qyb) + mu bn(vyp-vyb) + mu mg,
- ft = bt(vxp-vxb-r ozb) >= -mu fn = -mu k(qyp-qyb) - mu bn(vyp-vyb) - mu mg,
- qxb - qxp <= d,
- qxp - qxb <= d
'''
D3 = Polyhedron.from_bounds(xu_min, xu_max)
D3.add_inequality(
    np.array([[0., 1., 0., 0., -1., 0., 0., 0., 0., 0.]]),
    np.array([m*g/k])
)
D3.add_inequality(
    np.array([[0., k, 0., 0., -k, 0., bn, 0., 0., -bn]]),
    np.array([m*g])
)
D3.add_inequality(
    np.array([[0., mu*k, 0., 0., -mu*k, -bt, mu*bn, -r*bt, bt, -mu*bn]]),
    np.array([mu*m*g])
)
D3.add_inequality(
    np.array([[0., mu*k, 0., 0., -mu*k, bt, mu*bn, r*bt, -bt, -mu*bn]]),
    np.array([mu*m*g])
)
D3.add_inequality(
    np.array([[1., 0., 0., -1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
D3.add_inequality(
    np.array([[-1., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
assert D3.bounded
assert not D3.empty

'''
domain n.4, ball sliding left wrt paddle if:
- qyb <= qyp + mg/k,
- fn = k(qyp-qyb) + bn(vyp-vyb) + mg >= 0,
- ft = bt(vxp-vxb-r ozb) >= mu fn = mu k(qyp-qyb) + mu bn(vyp-vyb) + mu mg,
- qxb - qxp <= d,
- qxp - qxb <= d
'''
D4 = Polyhedron.from_bounds(xu_min, xu_max)
D4.add_inequality(
    np.array([[0., 1., 0., 0., -1., 0., 0., 0., 0., 0.]]),
    np.array([m*g/k])
)
D4.add_inequality(
    np.array([[0., k, 0., 0., -k, 0., bn, 0., 0., -bn]]),
    np.array([m*g])
)
D4.add_inequality(
    np.array([[0., -mu*k, 0., 0., mu*k, bt, -mu*bn, r*bt, -bt, mu*bn]]),
    np.array([-mu*m*g])
)
D3.add_inequality(
    np.array([[1., 0., 0., -1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
D3.add_inequality(
    np.array([[-1., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
assert D4.bounded
assert not D4.empty

'''
domain n.5, ball sliding right wrt paddle if:
- qyb <= qyp + mg/k,
- fn = k(qyp-qyb) + bn(vyp-vyb) + mg >= 0,
- ft = bt(vxp-vxb-r ozb) <= -mu fn = -mu k(qyp-qyb) - mu bn(vyp-vyb) - mu mg,
- qxb - qxp <= d,
- qxp - qxb <= d
'''
D5 = Polyhedron.from_bounds(xu_min, xu_max)
D5.add_inequality(
    np.array([[0., 1., 0., 0., -1., 0., 0., 0., 0., 0.]]),
    np.array([m*g/k])
)
D5.add_inequality(
    np.array([[0., k, 0., 0., -k, 0., bn, 0., 0., -bn]]),
    np.array([m*g])
)
D5.add_inequality(
    np.array([[0., -mu*k, 0., 0., mu*k, -bt, -mu*bn, -r*bt, bt, mu*bn]]),
    np.array([-mu*m*g])
)
D3.add_inequality(
    np.array([[1., 0., 0., -1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
D3.add_inequality(
    np.array([[-1., 0., 0., 1., 0., 0., 0., 0., 0., 0.]]),
    np.array([d])
)
assert D5.bounded
assert not D5.empty

# list of domains
D_list = [D1, D2, D3, D4, D5]

In [206]:
# state: x = (qxb, qyb, tzb, qxp, qyp, vxb, vyb, ozb)
# input: u = (vxp, vyp)

# PWA system
S = PieceWiseAffineSystem(S_list, D_list)

# controller parameters
N = 50
Q = np.diag([1.,1.,1.,1.,1.,.1,1.,1.])*h
R = np.diag([1.,1.])*h

# terminal set and cost
P = Q
X_N = Polyhedron.from_bounds(*[np.zeros(8)]*2)

In [207]:
controller = HybridModelPredictiveController(S, N, Q, R, P, X_N, method='Convex hull')
# controller.prog.setParam('Heuristics', 0)

In [208]:
# initial condition
x0 = np.array([0.,0.,2.*np.pi,0.,0.,0.,0.,0.])*.001

In [202]:
u, x, ms, cost = controller.feedforward(x0)

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 10980 rows, 5258 columns and 37180 nonzeros
Model has 508 quadratic objective terms
Variable types: 5008 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [2e-03, 1e+03]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-02, 1e-01]
  Bounds range     [6e-03, 6e-03]
  RHS range        [1e+00, 1e+00]
Presolve removed 4476 rows and 2095 columns
Presolve time: 0.08s
Presolved: 6504 rows, 3163 columns, 23144 nonzeros
Presolved model has 492 quadratic objective terms
Variable types: 2914 continuous, 249 integer (249 binary)

Root relaxation: infeasible, 13104 iterations, 0.21 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 infeasible    0               - infeasible      -     -    0s

Explored 0 nodes (13104 simplex iterations) in 0.35 second

In [ ]:
plot_input_sequence(u, h, (u_min, u_max))

In [ ]:
plot_state_trajectory(x, h, (x_min, x_max))

In [218]:
x0 = np.array([0.,.0,0.,0.,0.,.0,0.,.0])
x, ms = S.simulate(x0, [np.array([0.1,0.1])]*100)

# Animation

In [211]:
import meshcat
from meshcat.geometry import Box, Sphere, Cylinder, MeshLambertMaterial
from meshcat.animation import Animation
import meshcat.transformations as tf

In [212]:
vis = meshcat.Visualizer()
# vis.jupyter_cell()
vis.open()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


<Visualizer using: <meshcat.visualizer.ViewerWindow instance at 0x1130196c8> at path: <meshcat.path.Path object at 0x113028168>>

In [213]:
paddle_z = .05
paddle_x = .3
red = 0xff2222
blue = 0x2222ff
green = 0x22ff22
vis['paddle'].set_object(Box([paddle_x, d*2., paddle_z]), MeshLambertMaterial(color=red))
vis['ball'].set_object(Sphere(r), MeshLambertMaterial(color=blue))
vis['ball_orientation'].set_object(Cylinder(.01, r), MeshLambertMaterial(color=green))

In [219]:
anim = Animation()
for t in range(len(x)):
    with anim.at_frame(vis, t*h*30) as frame:
        frame['paddle'].set_transform(
            tf.translation_matrix([0, x[t][3], x[t][4]+m*g/k-paddle_z/2.])
        )
        frame['ball'].set_transform(
            tf.translation_matrix([0, x[t][0], x[t][1]+r])
        )
        frame['ball_orientation'].set_transform(
            tf.translation_matrix([0, x[t][0], x[t][1]+r]).dot(
                tf.rotation_matrix(x[t][2], [1.,0.,0.])
            )
        )
vis.set_animation(anim)